<a href="https://colab.research.google.com/github/markbojic/parallel-alg/blob/main/Projekat2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 18.0MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621085 sha256=04c13ab1dbbfaf89e69f04fc2a5cb469dfd2e36260993f843866d50edc2a09e2
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=8fa1ecf74a359ffd7fa27cb0698a1752e96fa4fc5a1f2f58f6991692724a6cab
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [ ]:
!nvidia-smi

Sat Jan 11 08:04:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import numpy as np
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit
import gc
import math

kernel_code_template = """
__global__ void MatrixMulKernel(int *A, int *B, int m, int n, int colsA,  int *C)
{
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;
    int sum = 0;

    if ((row < m) && (col < n)){
      for (int i = 0; i < colsA; ++i)
        sum += A[colsA*row + i] * B[i*n+col];
      C[row*n+col] = sum;
    }
}
"""

# Velicina matrica za mnozenje - Matrice dele MATRIX_SIZE_2
MATRIX_SIZE_1 = 100
MATRIX_SIZE_2 = 80
MATRIX_SIZE_3 = 100

# Generisanje 2 matrice odredjene velicine sa random elementima
a_cpu = np.random.randn(MATRIX_SIZE_1, MATRIX_SIZE_2).astype(np.int32)
b_cpu = np.random.randn(MATRIX_SIZE_2, MATRIX_SIZE_3).astype(np.int32)

# Prebacivanje matrica na gpu 
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)

# Prazna matrica istih dimenizja- Koristi se za cuvanje rezultata mnozenja
c_gpu = gpuarray.empty((MATRIX_SIZE_1, MATRIX_SIZE_3), np.int32)

c_cpu = np.random.randn(MATRIX_SIZE_1, MATRIX_SIZE_3).astype(np.int32)

# Kompajlovanje kernel koda u SOurceMOdule
mod = compiler.SourceModule(kernel_code_template)

# Uzmi kernel funkciju uz mod-a
matrixmul = mod.get_function("MatrixMulKernel")

#Konvertovanje dimenzija kako bi se poslalo u kernel funkciju posle
row1 = np.int32(MATRIX_SIZE_1)
col1 = np.int32(MATRIX_SIZE_2)
row2 = np.int32(MATRIX_SIZE_2)
col2 = np.int32(MATRIX_SIZE_3)

#pozovi kernel
matrixmul(
    # In
    a_gpu, b_gpu, row1, col2, col1, 
    # Out
    c_gpu, 
    #koliko thredova da koristi blok
    block = (32, 32, 1),
    grid = (math.ceil(c_cpu.shape[0]/32), math.ceil(c_cpu.shape[0]/32), 1)
    )


# Stampanje rezultata mnozenja
print ("Matrica A:")
print (a_gpu)

print( "\n")
print("Matrca B:")
print(b_gpu)

print( "\n")
print( "Matrica C:")
print(c_gpu)

print("\n")
print("Ocekivano:")
c_cpu = np.matmul(a_cpu, b_cpu)
print(c_cpu)
print("\n")


a_gpu.gpudata.free()
b_gpu.gpudata.free()
c_gpu.gpudata.free()
del a_gpu
del b_gpu
del c_gpu
del a_cpu
del b_cpu


#Zato sto zeza nakon nekoliko pustanja- javlja Illegal memory access (Logic Error), zato je gore i brisanje elemenata
#Force garbage Collector za svaki slucaj
gc.collect()

Matrica A:
[[ 0 -1  0 ...  0  0  2]
 [ 0 -3  0 ...  0  0  0]
 [-1  2  0 ...  0  0  1]
 ...
 [ 0  0  1 ...  0  0  0]
 [ 0  0 -1 ...  0 -1  0]
 [ 0  0  0 ...  0  0  0]]


Matrca B:
[[ 0  0  0 ...  0 -2  0]
 [ 0  0  0 ... -1  0  0]
 [ 0  0  0 ...  1  0  0]
 ...
 [ 2 -1  0 ...  1  0  0]
 [ 2 -2  1 ...  1  1  0]
 [ 0  0  0 ...  0  0  0]]


Matrica C:
[[-3 -4  1 ...  0 -5 -2]
 [ 6 -6 -3 ...  0 -5 -1]
 [-3  6 -2 ... -8 -2 -1]
 ...
 [-4 -5  0 ...  1 -4 -2]
 [-4  4  0 ...  6  2 -4]
 [ 1 -3 -1 ...  0 -2  3]]


Ocekivano:
[[-3 -4  1 ...  0 -5 -2]
 [ 6 -6 -3 ...  0 -5 -1]
 [-3  6 -2 ... -8 -2 -1]
 ...
 [-4 -5  0 ...  1 -4 -2]
 [-4  4  0 ...  6  2 -4]
 [ 1 -3 -1 ...  0 -2  3]]




3690